In [1]:
import sys
import os
sys.path.append("..")
from globals import ROOT_DIR
from data_providers import TextDataProvider
import configparser
import argparse
import configparser
from torch import optim
from experiment_builder import ExperimentBuilder
from data_providers import *
import os
from models.cnn import *
from models.multilayer_perceptron import multi_layer_perceptron
config = configparser.ConfigParser()
config.read('../config.ini')
path_data = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_DATA'])
path_labels = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_LABELS'])

In [2]:
# dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 
#            'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 
#            'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 
#            'user', 'geo', 'coordinates', 'place', 'contributors', 'is_quote_status', 
#            'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'possibly_sensitive', 'lang'])

def extract_tweets(data, filename, subset=None):
    start = time.time()
    print("=== Extracting tweets from JSON ===")
    tweets = []
    labels = []
    line_count = 0
    labels_map = {'hateful': 0, 'abusive': 1, 'normal': 2, 'spam': 3}
    error_count = 0
    outputs = []
    with open(filename, 'r') as f:
        for line in f.readlines():
            if subset is not None and line_count >= subset:
                break
            obj = json.loads(line)
            if data[obj['id_str']] not in labels_map:
                error_count += 1
                continue
            
            output = {}
            output['text'] = obj['text']
            output['label'] = labels_map[data[obj['id_str']]]
            labels.append(output['label'])
            output['retweet_count'] = obj['retweet_count']
            output['retweeted'] = int(obj['retweeted'])
            output['in_reply_to_status_id'] = obj['in_reply_to_status_id'] if obj['in_reply_to_status_id'] is not None else -1
            output['favorite_count'] = obj['favorite_count']
            output['label_string'] = data[obj['id_str']]
            outputs.append(output)
            
    print("Total time {}".format((time.time() - start)/60))
    return outputs, labels

In [3]:
def extract_labels(filename):
    print("=== Extracting annotations ===")
    data = {}
    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            data[row['tweet_id']] = row['maj_label']
    return data

In [4]:
path_data = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_DATA'])
path_labels = os.path.join(ROOT_DIR, config['DEFAULT']['PATH_LABELS'])
data = extract_labels(path_labels)
outputs, labels = extract_tweets(data, path_data)

=== Extracting annotations ===
=== Extracting tweets from JSON ===
Total time 0.04245690107345581


In [5]:
status_ids = [output['in_reply_to_status_id'] for output in outputs if output['in_reply_to_status_id'] != -1]
status_ids 

[848960211155111936,
 848683093322018817,
 847652506372984835,
 848974465681641472,
 850153748353646594,
 848644610108919808,
 850090043423281152,
 850450851084857345,
 848898137888620544,
 848738111378956288,
 848593367630045185,
 850354717544914944,
 847870832185663488,
 849401446097440768,
 850758182398021633,
 848271946362089473,
 849126476309864448,
 849159076995112961,
 848682175465754627,
 849960037313392642,
 850798998218842114,
 846965053240807424,
 850740104956186625,
 849362962452291586,
 849568312061083648,
 849078728823123968,
 847892319672184832,
 850196399513718785,
 850760996218179584,
 848840116214784001,
 849300802795687937,
 849585722680102913,
 848627032330125313,
 848340674533523456,
 850566695295082497,
 849679114416115714,
 850366375159791616,
 850294273337692160,
 849091192105172992,
 847971465043468288,
 846841493952319489,
 847873050439766016,
 847560758888206337,
 785661160200732672,
 848412362918871040,
 849434658970492928,
 850446173605441537,
 850162885800

In [6]:
len(status_ids)

11934

In [ ]:
# collected tweets from tweepy 
# start_ptr = 11800
# end_ptr = start_ptr + 100 
# while(start_ptr <= len(status_ids)):
#     print("Start ptr is at {}".format(start_ptr))
#     reply_tweets = api.statuses_lookup(status_ids[start_ptr:end_ptr],trim_user=True)
    
#     # load and save results each time 
#     replies = np.load(os.path.join(ROOT_DIR, 'data/reply_data.npy'))
#     replies = replies[()]
#     for i, reply_tweet in enumerate(reply_tweets):
#         reply_tweet = reply_tweet._json
#         replies[reply_tweet['id_str']] = reply_tweet['text']
#     np.save(os.path.join(ROOT_DIR, 'data/reply_data.npy'), replies)
    
#     start_ptr += 100 
#     end_ptr += 100

In [8]:
replies = np.load(os.path.join(ROOT_DIR, 'data/reply_data.npy'))
replies = replies[()]
count = 0 
# 0.16608010725657785 deleted... 
for status_id in status_ids:
    if str(status_id) not in replies:
        count += 1
print(count / len(status_ids))

0.16608010725657785


In [9]:
#verifies missing tweet 
# missing_tweet = api.statuses_lookup([847652506372984835],trim_user=True)
# missing_tweet

NameError: name 'api' is not defined

In [10]:
#combining tweets 
status_ids_fetched = []
outputs_context = []
for output in outputs:
    status_id = str(output['in_reply_to_status_id'])
    if status_id in replies:
        output['reply_to_tweet_text'] = output['text'] + replies[status_id]
    else:
        output['reply_to_tweet_text'] = output['text'] + output['text']
    outputs_context.append(output)

In [32]:
import string 

def tokenize(outpus):
    key = 'reply_to_tweet_text'
    outputs_processed = []
    for output in outputs:
        text = output[key]
        text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = text.split(' ')
        output['tokens'] = tokens
        outputs_processed.append(output)
    return outputs_processed

In [33]:
outputs_processed = tokenize(outputs_context)
outputs_processed[0]

{'text': 'fucks sake go away stupid anon — ^  https://t.co/8TQGyiKCVE',
 'label': 1,
 'retweet_count': 0,
 'retweeted': 0,
 'in_reply_to_status_id': -1,
 'favorite_count': 0,
 'label_string': 'abusive',
 'reply_to_tweet_text': 'fucks sake go away stupid anon — ^  https://t.co/8TQGyiKCVEfucks sake go away stupid anon — ^  https://t.co/8TQGyiKCVE',
 'tokens': ['fucks',
  'sake',
  'go',
  'away',
  'stupid',
  'anon',
  '—',
  '',
  '',
  'httpstco8TQGyiKCVEfucks',
  'sake',
  'go',
  'away',
  'stupid',
  'anon',
  '—',
  '',
  '',
  'httpstco8TQGyiKCVE']}

In [34]:
x_train, y_train, x_valid, y_valid, x_test, y_test = split_data(outputs_processed, labels, 28)
x_train[0]

[Sizes] Training set: 64.00%, Validation set: 16.00%, Test set: 20.00%


{'text': "#BravesAreBack let's go!!! #Braves #bravescountry @kapaya1234 you ready? Is the commute to SunTrust easier for you?",
 'label': 2,
 'retweet_count': 0,
 'retweeted': 0,
 'in_reply_to_status_id': -1,
 'favorite_count': 0,
 'label_string': 'normal',
 'reply_to_tweet_text': "#BravesAreBack let's go!!! #Braves #bravescountry @kapaya1234 you ready? Is the commute to SunTrust easier for you?#BravesAreBack let's go!!! #Braves #bravescountry @kapaya1234 you ready? Is the commute to SunTrust easier for you?",
 'tokens': ['BravesAreBack',
  'lets',
  'go',
  'Braves',
  'bravescountry',
  'kapaya1234',
  'you',
  'ready',
  'Is',
  'the',
  'commute',
  'to',
  'SunTrust',
  'easier',
  'for',
  'youBravesAreBack',
  'lets',
  'go',
  'Braves',
  'bravescountry',
  'kapaya1234',
  'you',
  'ready',
  'Is',
  'the',
  'commute',
  'to',
  'SunTrust',
  'easier',
  'for',
  'you']}

In [14]:
GOOGLE_EMBED_DIM = 300
TWITTER_EMBED_DIM = 400
TWEET_SENTENCE_SIZE = 17*2 # 16 is average tweet token length
TWEET_WORD_SIZE = 20 # selected by histogram of tweet counts
FASTTEXT_EMBED_DIM = 300
EMBED_DIM = 200
NUM_CLASSES = 4

def generate_random_embedding(embed_dim):
    return np.random.normal(scale=0.6, size=(embed_dim,))


# embeds tokens! 
def fetch_word_embeddings(outputs, word_vectors, embed_dim):
    outputs_embed = [] 
    for i, output in enumerate(outputs):
        tweet = output['tokens']
        embedded_tweet = []

        # trim if too large
        if len(tweet) >= TWEET_SENTENCE_SIZE:
            tweet = tweet[:TWEET_SENTENCE_SIZE]

        # convert all into word embeddings
        for word in tweet:
            embedding = generate_random_embedding(embed_dim) if word not in word_vectors else word_vectors[word]
            embedded_tweet.append(embedding)

        # pad if too short
        if len(tweet) < TWEET_SENTENCE_SIZE:
            diff = TWEET_SENTENCE_SIZE - len(tweet)
            embedded_tweet += [generate_random_embedding(embed_dim) for _ in range(diff)]

        assert len(embedded_tweet) == TWEET_SENTENCE_SIZE
        output['word_embeddings'] = embedded_tweet
        outputs_embed.append(output)
    return outputs_embed

In [15]:
start = time.time()
embed_dim = 400
filename = os.path.join(ROOT_DIR, 'data/word2vec_twitter_model/word2vec_twitter_model.bin')
word_vectors = KeyedVectors.load_word2vec_format(filename, binary=True, unicode_errors='ignore')
print("Total time {} min".format((time.time() - start) / 60))

Total time 0.7507729848225911 min


In [35]:
start = time.time()
x_train_embed = fetch_word_embeddings(x_train, word_vectors, embed_dim)
x_valid_embed = fetch_word_embeddings(x_valid, word_vectors, embed_dim)
x_test_embed = fetch_word_embeddings(x_test, word_vectors, embed_dim)
print("Total time {} min".format((time.time() - start) / 60))
print(len(x_train_embed), len(x_valid_embed), len(x_test_embed))

Total time 0.5721803188323975 min
37348 9338 11672


In [36]:
x_train_embed[0].keys() # list of dictionaries 

dict_keys(['text', 'label', 'retweet_count', 'retweeted', 'in_reply_to_status_id', 'favorite_count', 'label_string', 'reply_to_tweet_text', 'tokens', 'word_embeddings'])

In [37]:
y_train[0] #int 

2

In [50]:
# 17 n-gram 
# retweet count 
# in reply to status id 
# favorite count 

# 34 n-gram 
def convert_to_feature_embeddings(x_embed):
    return [x['word_embeddings'] for x in x_embed]
 
data = {}
print(x_train_embed[0].keys())
data['x_train'] = convert_to_feature_embeddings(x_train_embed)
data['y_train'] = y_train
data['x_valid'] = convert_to_feature_embeddings(x_valid_embed)
data['y_valid'] = y_valid
data['x_test'] = convert_to_feature_embeddings(x_test_embed)
data['y_test'] = y_test


dict_keys(['text', 'label', 'retweet_count', 'retweeted', 'in_reply_to_status_id', 'favorite_count', 'label_string', 'reply_to_tweet_text', 'tokens', 'word_embeddings'])


In [39]:
from torchvision import transforms

def wrap_data(batch_size, seed, x_train, y_train, x_valid, y_valid, x_test, y_test):
    transform = transforms.Compose([
        transforms.ToTensor(),
    ])
    transform=None
    
    train_set = DataProvider(inputs=x_train, targets=y_train, seed=seed, transform=transform)
    train_data_local = torch.utils.data.DataLoader(train_set,
                                                   batch_size=batch_size,
                                                   num_workers=2,
                                                   sampler=ImbalancedDatasetSampler(train_set),
                                                   )

    valid_set = DataProvider(inputs=x_valid, targets=y_valid, seed=seed, transform=transform)
    valid_data_local = torch.utils.data.DataLoader(valid_set,
                                                   batch_size=batch_size,
                                                   num_workers=2,
                                                   shuffle=False,
                                                  )

    test_set = DataProvider(inputs=x_test, targets=y_test, seed=seed, transform=transform)
    test_data_local = torch.utils.data.DataLoader(test_set,
                                                  batch_size=batch_size,
                                                  num_workers=2,
                                                  shuffle=False,
                                                 )
    return train_data_local, valid_data_local, test_data_local

def fetch_model(model, embedding_level, input_shape_local, dropout):
    if model == 'MLP':
        return multi_layer_perceptron(input_shape_local)
    if model == 'CNN':
        if embedding_level == 'word':
            return word_cnn(input_shape_local, dropout)
        elif embedding_level == 'character':
            return character_cnn(input_shape_local)
    if model == 'DENSENET':
        return densenet()
    else:
        raise ValueError("Model key not found {}".format(embedding_level))


def fetch_model_parameters(input_shape_local):
    model_local = fetch_model(model='CNN',
                            embedding_level='word',
                            input_shape_local=input_shape_local,
                            dropout=0.5)
    criterion_local = torch.nn.CrossEntropyLoss()
    optimizer_local = torch.optim.Adam(model_local.parameters(), weight_decay=1e-4)
    scheduler_local = optim.lr_scheduler.CosineAnnealingLR(optimizer_local, T_max=100, eta_min=0.0001)
    return model_local, criterion_local, optimizer_local, scheduler_local



In [40]:
train_data, valid_data, test_data = wrap_data(2048, 28, **data)

In [41]:
for x,y in train_data:
    input_shape = x.shape
    break 

In [43]:
input_shape = tuple(input_shape)
input_shape

(2048, 34, 400)

In [44]:
def run_train_iter(model, device, optimizer, criterion, x, y, stats, experiment_key='train'):
    """
    Receives the inputs and targets for the model and runs a training iteration. Returns loss and accuracy metrics.
    :param x: The inputs to the model. A numpy array of shape batch_size, channels, height, width
    :param y: The targets for the model. A numpy array of shape batch_size, num_classes
    :return: the loss and accuracy for this batch
    """
    # sets model to training mode
    # (in case batch normalization or other methods have different procedures for training and evaluation)
    model.train()
    x = x.float()
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()  # set all weight grads from previous training iters to 0
    out = model.forward(x)  # forward the data in the model
    # loss = F.cross_entropy(input=out, target=y)  # compute loss
    loss = criterion(out, y)
    loss.backward()  # backpropagate to compute gradients for current iter loss

    optimizer.step()  # update network parameters
    _, predicted = torch.max(out.data, 1)  # get argmax of predictions
    accuracy = np.mean(list(predicted.eq(y.data).cpu()))  # compute accuracy
    stats['{}_acc'.format(experiment_key)].append(accuracy)
    stats['{}_loss'.format(experiment_key)].append(loss.data.detach().cpu().numpy())

def run_evaluation_iter(model, device, optimizer, criterion, x, y, stats, experiment_key='valid'):
    """
    Receives the inputs and targets for the model and runs an evaluation iterations. Returns loss and accuracy metrics.
    :param x: The inputs to the model. A numpy array of shape batch_size, channels, height, width
    :param y: The targets for the model. A numpy array of shape batch_size, num_classes
    :return: the loss and accuracy for this batch
    """
    with torch.no_grad():
        model.eval()  # sets the system to validation mode
        x = x.float()
        x = x.to(device)
        y = y.to(device)
        out = model.forward(x)  # forward the data in the model
        loss = criterion(out, y)
        
        # loss = F.cross_entropy(out, y)  # compute loss
        _, predicted = torch.max(out.data, 1)  # get argmax of predictions
        
        accuracy = np.mean(list(predicted.eq(y.data).cpu()))
        stats['{}_acc'.format(experiment_key)].append(accuracy)  # compute accuracy
        stats['{}_loss'.format(experiment_key)].append(loss.data.detach().cpu().numpy())
        

In [45]:
def save_model(model, model_save_dir, model_save_name, model_idx):
    """
    Save the network parameter state and current best val epoch idx and best val accuracy.
    :param model_save_name: Name to use to save model without the epoch index
    :param model_idx: The index to save the model with.
    :param best_validation_model_idx: The index of the best validation model to be stored for future use.
    :param best_validation_model_acc: The best validation accuracy to be stored for use at test time.
    :param model_save_dir: The directory to store the state at.
    :param state: The dictionary containing the system state.

    """
    # Save state each epoch
    path = os.path.join(model_save_dir, "{}_{}".format(model_save_name, str(model_idx)))
    torch.save(model.state_dict(), f=path)
    

def load_model(model, model_save_dir, model_save_name, model_idx):
    """
    Load the network parameter state and the best val model idx and best val acc to be compared with the future val accuracies, in order to choose the best val model
    :param model_save_dir: The directory to store the state at.
    :param model_save_name: Name to use to save model without the epoch index
    :param model_idx: The index to save the model with.
    """
    path = os.path.join(model_save_dir, "{}_{}".format(model_save_name, str(model_idx)))
    checkpoint = torch.load(f=path)
    # freeze parameters
    model.load_state_dict(checkpoint)
    for parameter in model.parameters():
        parameter.requires_grad = False
    return model 

In [46]:
from collections import OrderedDict, defaultdict
import tqdm

model, criterion, optimizer, _ = fetch_model_parameters(input_shape)
device = torch.device('cpu')
train_stats = OrderedDict()
num_epochs = 2

for epoch_idx in range(num_epochs):
    epoch_start_time = time.time()
    epoch_stats = defaultdict(list)
    with tqdm.tqdm(total=len(train_data)) as pbar_train:  # create a progress bar for training
        for idx, (x, y) in enumerate(train_data):  # get data batches
            run_train_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats)  # take a training iter step
            pbar_train.update(1)
            pbar_train.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['train_loss'][-1],
                                                                               epoch_stats['train_acc'][-1]))

    with tqdm.tqdm(total=len(valid_data)) as pbar_val:  # create a progress bar for validation
        for x, y in valid_data:  # get data batches
            run_evaluation_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats)  # run a validation iter
            pbar_val.update(1)  # add 1 step to the progress bar
            pbar_val.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['valid_loss'][-1],
                                                                             epoch_stats['valid_acc'][-1]))
     
    
    
    save_model(model, '', 'testing', epoch_idx)

Building basic block of ConvolutionalNetwork using input shape torch.Size([2048, 400, 34])


  0%|          | 0/19 [00:00<?, ?it/s]

Block is built, output volume is torch.Size([2048, 4])


loss: 1.3163, accuracy: 0.2679: 100%|██████████| 5/5 [00:09<00:00,  1.81s/it]


In [47]:
loaded_model = load_model(model, '', 'testing', 1)

#evaluate test here
with tqdm.tqdm(total=len(test_data)) as pbar_test:  # create a progress bar for validation
    for x, y in test_data:  # get data batches
        run_evaluation_iter(model, device, optimizer, criterion, x=x, y=y, stats=epoch_stats, experiment_key="test_experiment")  # run a validation iter
        pbar_test.update(1)  # add 1 step to the progress bar
        pbar_test.set_description("loss: {:.4f}, accuracy: {:.4f}".format(epoch_stats['test_experiment_loss'][-1],
                                                                  epoch_stats['test_experiment_acc'][-1]))


loss: 1.2806, accuracy: 0.3052: 100%|██████████| 6/6 [00:08<00:00,  1.23s/it]


In [ ]:
epoch_stats